###Installing and uploading the needed packages in our project

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip install sentencepiece
import transformers
import pandas as pd
import numpy as np
import random
from huggingface_hub import notebook_login
from transformers.utils import send_example_telemetry
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, load_metric
from IPython.display import display, HTML
from transformers import AutoTokenizer
from keras.callbacks import ModelCheckpoint
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [ ]:
#notebook_login()

In [ ]:
#send_example_telemetry("translation_notebook", framework="pytorch")

###Defining our Model: Flan T5 Base

In [ ]:
#Defining our model :  Flan T5 Base
model_checkpoint = "google/flan-t5-base"


###Loading the dataset

In [ ]:
#loading dataset" Dora it contains more than 10 000 rows
df =  pd.read_csv ('./sentences_.csv')
metric = load_metric("sacrebleu")

In [ ]:
#Dropping the null rows
df = df.dropna()
#Trying to reduce the overfitting by addin a string to out 'english column
df['english'] = 'translate to arabic : ' + df['english'].astype(str)
df['english'].head()

###Spliting data to 2 parts : 80% for training, 20% for test

In [ ]:
# split the data into train and test set
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

###Hugging face dataset load from CSV

In [ ]:
#Hugging face dataset load from CSV
train_ = Dataset.from_pandas(train)
test_ = Dataset.from_pandas(test)
#Transforming our dataset to the hugging face dictset format
df = DatasetDict()
#Removing a generated Index column
df.remove_columns("__index_level_0__")
df['train'] = train_.remove_columns("__index_level_0__")
df['test'] = test_.remove_columns("__index_level_0__")


###Generalizing random data 

In [ ]:
#Showing some random examples from our dataset t understand what our dataset looks like
def show_random_elements(df, num_examples=5):
    assert num_examples <= len(df), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(df)-1)
        while pick in picks:
            pick = random.randint(0, len(df)-1)
        picks.append(pick)
    
    data = pd.DataFrame(df[picks])
    for column, typ in df.features.items():
        if isinstance(typ, datasets.ClassLabel):
            data[column] = data[column].transform(lambda i: typ.names[i])
    display(HTML(data.to_html()))

In [ ]:
show_random_elements(df["train"])

,english,darija
0,translate to arabic : Or are you just saying that to make fun of me?,olla tatgoul dakchi hir bach tD7ek 3lia?
1,translate to arabic : I can't think of anything worse than a rubbish band playing all night,man9drch nfakar fchi haja khyeb mn jo9 dya zbal kayghnniw llil kaml
2,translate to arabic : He is crying,kaybki
3,translate to arabic : I want to leave,bghit nmchi f7alli
4,"translate to arabic : Careful, there's roots everywhere","reddo lbal, rah kaynin jjdoura finmma mchiti"


###Generating our metric

In [ ]:
#Generating a matric
metric
fake_preds = ["hello brother", "salam khouya"]
fake_labels = [["hello brother"], ["salam khouya"]]
metric.compute(predictions=fake_preds, references=fake_labels)

###Preprocessing data
we need to preprocess our data before giving it to our model. 
We used Transformers Tokenizer tokenize the inputs.

In [ ]:
#Preprocessing data   
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ar_AR"

In [ ]:
#testing out tokenizer
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#we need to tokenize them inside the as_target_tokenizer context manager. 
#This will make sure the tokenizer uses the special tokens corresponding to the targets
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

In [ ]:
#T5 checkpoints require a special prefix to put before the inputs
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to darija: "
else:
    prefix = ""

In [ ]:
#the function that will preprocess our samples
max_input_length = 128
max_target_length = 128
source_lang = "english"
target_lang = "darija"

def preprocess_function(examples):
    print(examples)
    inputs = [prefix + ex  for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
show_random_elements(df["train"])

,english,darija
0,translate to arabic : Did you eat any of those mushrooms?,wach kliti chi w7da mn had champignons?
1,translate to arabic : Don't overheat them.,matskhenhomch bzzaf.
2,translate to arabic : we need solution,khasna 7all
3,"translate to arabic : Yes, of course!",ah darori!
4,translate to arabic : I am just pulling your leg,kent ghir kantfella m3ak


To apply the preprocess function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier

In [ ]:
tokenized_datasets = df.map(preprocess_function, batched=True)


###Finetuning

In [ ]:
#Finetuning
#we use the AutoModelForSeq2SeqLM class. Like with the tokenizer.
#the from_pretrained method will download and cache the model for us.
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

To instantiate a Seq2SeqTrainer, we will need to define three more things. The most important is the Seq2SeqTrainingArguments, which is a class that contains all the attributes to customize the training. 

In [ ]:
source_lang = "english"
target_lang = "darija"
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions.

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

we need to pass all of this along with our datasets to the Seq2SeqTrainer

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the train method

In [ ]:

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,4.092500,3.481138,0.515200,15.920500
2,3.545600,3.240424,0.640900,15.955000
3,3.372800,3.116794,0.987900,16.162000
4,3.279900,3.058258,1.006300,16.069000
5,3.244500,3.039718,0.890300,16.235000


TrainOutput(global_step=2500, training_loss=3.507073388671875, metrics={'train_runtime': 1359.4898, 'train_samples_per_second': 29.408, 'train_steps_per_second': 1.839, 'total_flos': 1355455780503552.0, 'train_loss': 3.507073388671875, 'epoch': 5.0})

###Testing with new data

In [ ]:
inputs = tokenizer("translate to arabic : Did you eat any of those mushrooms?", return_tensors="pt").to("cuda")
output = model.generate(**inputs)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0,     3, 10674,     3,    29,    17,     9,     3,    40,    23,
             9,     3,    40,   519,     9,    26,     3,    40,   519,     9]],
       device='cuda:0')

In [ ]:
tokenizer.decode(output.cpu().numpy()[0])

'<pad> wach nta lia l3ad l3a'